In [4]:
import sounddevice as sd
from scipy.io.wavfile import write
import torch
import torchaudio
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
import numpy as np
from pydub import AudioSegment
import requests
import json
import spacy
import csv


/Users/nerve/Coding/Hackathon/voice/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [17]:
import requests
import os
from pydub import AudioSegment

# Step 1: Simulate someone uploading a file to the Flask server
# Note: This step should be performed by the client, not in the notebook

# Step 2: Wait for the file to be uploaded and retrieve it from the server

file_name = 'newupload.wav'  # Replace with the actual file name
file_path = os.path.join(file_name)
print(file_path)

# Check if the file exists
if os.path.exists(file_path):
    # Load the file in the Jupyter Notebook
    audio = AudioSegment.from_wav(file_path)
    print(f"Loaded file from {file_path}")
else:
    print(f"File {file_name} not found")

newupload.wav
Loaded file from newupload.wav


In [18]:
# Load the model and processor
model_name = "facebook/wav2vec2-base-960h"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)

# Load your uploaded audio file using pydub

audio = AudioSegment.from_wav(file_path)

# Convert pydub audio to numpy array
speech_array = np.array(audio.get_array_of_samples(), dtype=np.float32)
sampling_rate = audio.frame_rate

# Normalize the audio array
speech_array = speech_array / np.iinfo(np.int16).max

# Ensure the audio is mono
if audio.channels > 1:
    speech_array = speech_array.reshape((-1, audio.channels)).mean(axis=1)

# Resample to 16000 Hz if necessary
if sampling_rate != 16000:
    from scipy.signal import resample
    speech_array = resample(speech_array, int(16000 * len(speech_array) / sampling_rate))
    sampling_rate = 16000

# Process the audio file
input_values = processor(speech_array, sampling_rate=sampling_rate, return_tensors="pt").input_values

# Perform the transcription
with torch.no_grad():
    logits = model(input_values).logits

# Decode the predicted ids to text
predicted_ids = torch.argmax(logits, dim=-1)
transcription = processor.decode(predicted_ids[0])

print("Transcription: ", transcription)

/Users/nerve/Coding/Hackathon/voice/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (ini

Transcription:  THE STALE SMELL OF OLD BEER LINGERS IT TAKES HEAT TO BRING OUT THE ODOR A COLD DIP RESTORES HEALTH AND ZEST A SALT PICKLE TASTES FINE WITH HAM TAKOZAL PASTORE ARE MY FAVORITE A ZESTFUL FOOD IS THE HOT CROSS BUN


In [39]:
# Example API request (use your own request method and URL as necessary)
url = "http://localhost:11434/api/generate"
data = {
    "model": "gemma2:latest",
    "prompt": transcription + " - correct this if there are spelling errors return only the corrected text",
    "stream": False
}

response = requests.post(url, json=data)

# Parse the JSON response
response_json = response.json()

# Extract the relevant response content (ignore the metadata)
model_response = response_json.get("response", "No response found")

# Print the cleaned response
print("Model Response:", model_response)

model_response = transcription

Model Response: The right tire is not okay. 


Let me know if you have any other phrases you'd like me to check! 😊  



In [5]:
# Load the small English model in spaCy
nlp = spacy.load("en_core_web_sm")

transcription = "This is not okay"

# Define the "okay" words lexicon
okay_words = set([
    "okay", "all right", "working", "good", "fine", "no issues",
    "everything is okay", "all clear", "no problem", "in order",
    "smooth", "operational", "functioning", "on track", "intact", 
    "secure", "stable", "sound"
])

# Define a simple negation lexicon
negation_words = set(["not", "no", "never", "none", "nobody", "nothing"])

# Process the transcription with spaCy
doc = nlp(transcription.lower())

# Tokenize the transcription and check for negation and okay words
tokens = [token.text for token in doc]
contains_okay_word = any(word in okay_words for word in tokens)
contains_negation = any(word in negation_words for word in tokens)


# Analyze the response based on lexicons
if contains_negation and contains_okay_word:
    analysis_result = "The statement might be negative despite containing 'okay' words."
    label = 1
elif contains_okay_word:
    analysis_result = "The statement is positive."
    label = 0
else:
    analysis_result = "The statement is negative or unclear."
    label = 1

# Print the analysis result
print("Analysis Result:", analysis_result)
print(label)



Analysis Result: The statement might be negative despite containing 'okay' words.
1


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the HF_TOKEN and SECRET_KEY
hf_token = os.getenv("HF_TOKEN")
secret_key = os.getenv("SECRET_KEY")

# Use hf_token for authentication if needed

SQL CONNECTION

In [1]:
import psycopg2
from psycopg2 import sql

# Database connection credentials
connection_params = {
    "user": "postgres.keqffvaytvbaqniavqtg",
    "host": "aws-0-us-east-1.pooler.supabase.com",
    "database": "postgres",
    "password": "9ChJhmA5BIkZUvr1",
    "port": 6543,  # Default port for Supabase Postgres
    "sslmode": "require"
}

try:
    # Establish a connection to the PostgreSQL database
    connection = psycopg2.connect(**connection_params)

    # Create a cursor object to interact with the database
    cursor = connection.cursor()

    # Example query
    cursor.execute("SELECT NOW();")

    # Fetch and print the result of the query
    result = cursor.fetchone()
    print("Current Time:", result)

except Exception as e:
    print("Error connecting to the database:", e)

finally:
    # Always close the cursor and connection to free up resources
    if cursor:
        cursor.close()
    if connection:
        connection.close()

Current Time: (datetime.datetime(2024, 8, 10, 6, 54, 18, 732791, tzinfo=datetime.timezone.utc),)


In [48]:
%reset -f


In [ ]:
# Parameters
duration = 8  # seconds
sample_rate = 44100  # Sample rate

print("Recording...")
# Record audio in mono
audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype='int16')
sd.wait()  # Wait until the recording is finished
print("Recording finished!")

# Save the recording as a WAV file
wav_file = "output.wav"
write(wav_file, sample_rate, audio_data)

print(f"Audio saved as {wav_file}")

# If you want to store the WAV data as a variable
audio_variable = audio_data